# TF-IDF
### Задание:
Составление словарей стоп-слов и тематических словарей с использованием tf/idf и контрастного метода.

**TF-IDF** (от англ. TF — term frequency, IDF — inverse document frequency) — статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса. Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции.

Мера TF-IDF часто используется в задачах анализа текстов и информационного поиска, например, как один из критериев релевантности документа поисковому запросу, при расчёте меры близости документов при кластеризации.

In [1]:
import codecs
import re
import math
import nltk
import collections
from nltk.stem import PorterStemmer, WordNetLemmatizer

### Функции используемые в работе:


In [2]:
#Токенизация текста
def tokenization(text):
    
    doc = re.split(r' *[\;\:\—\<\>\\\"\.\,\!\?\&\=\+\(\)\{\}\[\]\r\n\t\« \»\“\”\„\/\d ]', text) 
    tokens = []
    for token in doc: 
        if token != '':
            tokens.append(token.lower())        
    return tokens

In [3]:
#считаем tf
def  tf(tokens):
    tf_countter = collections.Counter(tokens)
    for i in tf_countter:
        tf_countter[i] = tf_countter[i]/float(len(tokens))
    return tf_countter

In [4]:
#считаем idf
def idf(word, corpus):
    return math.log(len(corpus)/sum([1.0 for text in corpus if word in text]))

In [5]:
#токенизируем каждый текст в корпусе
def tokenization_corpus(corpus):
    corpus_tokins = []
    for text in corpus:
        corpus_tokins.append(tokenization(text))
    return corpus_tokins

In [6]:
# получаем все токены в корпусе
def all_corpus_tokens(corpus_tokens):
    all_tokens =[]
    for text in corpus_tokens:
        for word in text:
            all_tokens.append(word)
    return all_tokens

In [7]:
#стемминг и лемматизация для одного текста
def stem_lem(tokens):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    return [stemmer.stem(lemmatizer.lemmatize(word)) for word in tokens]
    #return [stemmer.stem(word) for word in tokens]
    #return [lemmatizer.lemmatize(word) for word in tokens]

In [8]:
#стемминг и лемматизация для корпуса текстов
def stem_lem_corpus(corpus):
    u_corp = []
    for tokens in corpus:
        u_corp.append(stem_lem(tokens))
    return u_corp

In [9]:
#выводим топ слов по tf-idf
def topN_tfidf(path,n=50):
    texts = []
    #получаем каждый документ и создаем корпус
    for i in range(10):

        fileObj = codecs.open( path + str(i)+'.txt', "r", "utf_8_sig" )
        text = fileObj.read() # или читайте по строке
        texts.append(text)
        fileObj.close()
    
    #Токенизация
    tokens_corp = tokenization_corpus(texts)
    all_tokens = all_corpus_tokens(tokens_corp)
    
    
    #Лемматизация и стемминг
    st_lm_all_tokkens = stem_lem(all_tokens)
    st_lm_tokens_corp = stem_lem_corpus(tokens_corp)
    
    #вычисляем tfidf и заносим в словарь
    tfidf = {}
    tf_counter = tf(st_lm_all_tokkens)
    for word in tf_counter:  
        tfidf[word] = tf_counter[word] * idf(word,st_lm_tokens_corp)
    
    #сортрировка словоря по возрастанию значений и возврат ТОП N(параметр) слов
    list_tfidf = list(tfidf.items())
    list_tfidf.sort(key=lambda i: i[1])
    return(list_tfidf[0:n])

In [10]:
#стоп-слова между двумя словарями
def stop_words(top_tfidf_1,top_tfidf_2):
    u_list = []
    for item1 in top_tfidf_1:
        for item2 in top_tfidf_2:
            if item1[0]==item2[0]:
                u_list.append(item1[0])
                break
    return u_list

In [11]:
#стоп-слова между тремя словарями
def stop_words_3(top_tfidf_CV,top_tfidf_FL,top_tfidf_FB):
    u_list2 = []
    for item1 in top_tfidf_CV:
        for item2 in top_tfidf_FL:
            if item1[0]==item2[0]:
                for item3 in top_tfidf_FB:
                    if item1[0]==item3[0]:
                        u_list2.append(item1[0])
                        break
                break
    return u_list2


In [12]:
#термины для top_tfidf_main
def unique_words(top_tfidf_main,top_tfidf_1,top_tfidf_2):
    unique_words = []
    for word in top_tfidf_main:
        if (word[0] in [w[0] for w in top_tfidf_1] ) == False:
            if (word[0] in [w[0] for w in top_tfidf_2] ) == False:
                unique_words.append(word)
    return unique_words

### TF-IDF словари 

In [13]:
pathCV = './example1/' #Computer vision articles
pathFB = './example2/' #Football history articles
pathFL = './example3/' #Flowers articles
N =50
top_tfidf_CV = topN_tfidf(pathCV,N)
top_tfidf_FB = topN_tfidf(pathFB,N)
top_tfidf_FL = topN_tfidf(pathFL,N)

In [14]:
print(top_tfidf_CV)
print(top_tfidf_FB)
print(top_tfidf_FL)

[('comput', 0.0), ('vision', 0.0), ('is', 0.0), ('that', 0.0), ('understand', 0.0), ('imag', 0.0), ('or', 0.0), ('the', 0.0), ('of', 0.0), ('it', 0.0), ('to', 0.0), ('and', 0.0), ('visual', 0.0), ('process', 0.0), ('extract', 0.0), ('data', 0.0), ('in', 0.0), ('thi', 0.0), ('into', 0.0), ('be', 0.0), ('a', 0.0), ('use', 0.0), ('mani', 0.0), ('object', 0.0), ('recognit', 0.0), ('by', 0.0), ('what', 0.0), ('ha', 0.0), ('are', 0.0), ('build', 8.412241096861635e-05), ('scienc', 0.00011877551036046723), ('sub-domain', 0.00012256268126864566), ('servo', 0.00012256268126864566), ('link', 0.00012256268126864566), ('basi', 0.00012256268126864566), ('meant', 0.00012256268126864566), ('stone', 0.00012256268126864566), ('endow', 0.00012256268126864566), ('attach', 0.00012256268126864566), ('preval', 0.00012256268126864566), ('foundat', 0.00012256268126864566), ('non-polyhedr', 0.00012256268126864566), ('polyhedr', 0.00012256268126864566), ('rigor', 0.00012256268126864566), ('quantit', 0.0001225626

### Стоп-слова
Попарно для каждого tf-idf словаря и для всех 3-х словарей вместе

In [15]:
stop_words(top_tfidf_CV,top_tfidf_FB)

['is', 'that', 'the', 'of', 'it', 'to', 'and', 'in', 'thi', 'be', 'a', 'by']

In [16]:
stop_words(top_tfidf_CV,top_tfidf_FL)

['is',
 'that',
 'or',
 'the',
 'of',
 'it',
 'to',
 'and',
 'in',
 'thi',
 'into',
 'be',
 'a',
 'by',
 'ha',
 'are']

In [17]:
stop_words(top_tfidf_FB,top_tfidf_FL)

['is',
 'a',
 'of',
 'that',
 'to',
 'the',
 'in',
 'and',
 'be',
 'at',
 'by',
 'for',
 'with',
 'more',
 'from',
 'it',
 'thi',
 'been',
 'one',
 'can']

In [18]:
stop_words_3(top_tfidf_CV,top_tfidf_FL,top_tfidf_FB)

['is', 'that', 'the', 'of', 'it', 'to', 'and', 'in', 'thi', 'be', 'a', 'by']

### Термины: 

In [19]:
unique_words_FB = unique_words(top_tfidf_FB,top_tfidf_FL,top_tfidf_CV)
unique_words_FL = unique_words(top_tfidf_FL,top_tfidf_FB,top_tfidf_CV)
unique_words_CV = unique_words(top_tfidf_CV,top_tfidf_FB,top_tfidf_FL)

In [20]:

print(unique_words_FB)
print(unique_words_FL)
print(unique_words_CV)

[('footbal', 0.0), ('most', 0.0), ('game', 0.0), ('were', 0.0), ('their', 0.0), ('wa', 0.0), ('first', 0.0), ('two', 0.0), ('right', 0.0), ('have', 0.0), ('who', 0.0), ('had', 0.0), ('up', 0.0), ('than', 0.0), ('final', 0.0), ('win', 0.0), ('year', 0.0), ('did', 5.092750293619255e-05), ('now', 5.9415420092224644e-05), ('remain', 6.365937867024068e-05), ('end', 7.214729582627278e-05), ('old', 8.063521298230488e-05), ('almost', 8.089470562426037e-05), ('home', 8.912313013833695e-05), ('came', 8.912313013833695e-05), ('commonli', 9.274893631652484e-05), ('contemporari', 9.274893631652484e-05), ('deliber', 9.274893631652484e-05), ('incorpor', 9.274893631652484e-05), ('tabletop', 9.274893631652484e-05)]
[('flower', 0.0), ('plant', 0.0), ('also', 0.0), ('produc', 0.0), ('contain', 0.0), ('on', 0.0), ('they', 0.0), ('speci', 0.0), ('which', 0.0), ('an', 0.0), ('but', 0.0), ('where', 8.560675657755544e-05), ('through', 8.988709440643321e-05), ('divis', 9.354398102758666e-05), ('magnoliophyta',